In [8]:
import os
import sys
import math
import json
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
from tqdm import tqdm_notebook as tqdm, tnrange
from transformers import BertModel, BertTokenizer
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/work'

In [2]:
from fgc_support_retri import config
from fgc_support_retri.sup_model import BertContextSupModel_V3
from fgc_support_retri.fgc_preprocess import BertV3Idx, SerContextDataset, bert_collate_v3
from fgc_support_retri.utils import read_fgc
from fgc_support_retri.eval import evalaluate_f1

I1217 01:44:36.571458 140111562508096 corenlp.py:42] Using an existing server http://140.109.19.191:9000
I1217 01:44:37.574596 140111562508096 corenlp.py:118] The server is available.


In [16]:
items = read_fgc(config.FGC_TEST, eval=True)

no gold supporting evidence
{'QID': 'D009Q03', 'QTYPE': '申论', 'QTEXT': '「佔領華爾街」運動的訴求為何?', 'SENTS': [{'text': '「占领华尔街」运动的诉求为何?', 'start': 0, 'end': 15}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'ATEXT_CN': ''}], 'ATYPE': 'Object', 'AMODE': 'Single-Span-Extraction', 'ASPAN': [], 'SHINT': [], 'QTEXT_CN': '「占领华尔街」运动的诉求为何?'}
no gold supporting evidence
{'QID': 'D073Q12', 'QTYPE': '申论', 'QTEXT': '台灣對於鯨鯊保育做了哪些努力?', 'SENTS': [{'text': '台湾对于鲸鲨保育做了哪些努力?', 'start': 0, 'end': 15}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'ATEXT_CN': ''}], 'ATYPE': 'Object', 'AMODE': 'Single-Span-Extraction', 'ASPAN': [], 'SHINT': [], 'QTEXT_CN': '台湾对于鲸鲨保育做了哪些努力?'}
no gold supporting evidence
{'QID': 'D245Q05', 'QTYPE': '申论', 'QTEXT': '為何聖伯多祿大殿只能重建不能整修就好?', 'SENTS': [{'text': '为何圣伯多禄大殿只能重建不能整修就好?', 'start': 0, 'end': 19}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'ATEXT_CN': ''}], 'ATYPE': 'Event', 'AMODE': 'Single-Span-Extraction', 'ASPAN': [], 'S

In [4]:
len(items)

90000

In [5]:
items[0]

{'QID': '5a7a06935542990198eaf050q01',
 'SENTS': [{'text': '《亚瑟》杂志（1844-1846）是19世纪在费城出版的美国文学期刊。',
   'start': 0,
   'end': 35},
  {'text': '它由T.S.亚瑟编辑，由埃德加A.坡、J.H.英格拉姆、莎拉·约瑟法·黑尔、托马斯·G.斯皮尔和其他人的作品组成。',
   'start': 35,
   'end': 127},
  {'text': '1846年5月，这本书被合并成《戈迪夫人的书》。', 'start': 127, 'end': 243},
  {'text': '首先是美国鲍尔传媒集团出版的女性杂志。', 'start': 243, 'end': 378},
  {'text': '这本杂志创办于1989年。', 'start': 378, 'end': 526},
  {'text': '总部位于新泽西州的恩格尔伍德克利夫。', 'start': 526, 'end': 692},
  {'text': '2011年该杂志的发行量为1310696份。', 'start': 692, 'end': 880}],
 'SUP_EVIDENCE': [0, 3],
 'QTEXT': '哪本杂志是第一本亚瑟的杂志还是第一本女性杂志？',
 'ANS': '亚瑟的杂志',
 'ASPAN': []}

In [7]:
items_dev = read_fgc(config.HOTPOT_DEV, eval=True)

KeyError: 'SENTS'

In [9]:
with open(config.HOTPOT_DEV) as f:
    items_dev = json.load(f)

In [10]:
len(items_dev)

7405

In [11]:
items_dev[0]

{'DID': '5a8b57f25542995d1e6f1371',
 'DTEXT': '斯科特·德里克森（1966年7月16日出生）是美国导演、编剧和制片人。\n他住在加州洛杉矶。\n他最出名的是执导恐怖电影，如《险恶》、《艾米丽·罗斯的驱魔》、《让我们远离邪恶》，以及2016年惊奇电影宇宙版的《奇怪医生》\n爱德华·戴维斯·伍德（1924年10月10日至1978年12月10日）是美国电影制片人、演员、作家、制片人和导演。\n',
 'QUESTIONS': [{'QID': '1',
   'QTEXT': '斯科特·德里克森和埃德·伍德是同一国籍的吗？',
   'QTYPE': '进阶题',
   'AMODE': 'Comparing-Members',
   'SHINT': [{'text': '斯科特·德里克森（1966年7月16日出生）是美国导演、编剧和制片人。',
     'start': 0,
     'end': 35},
    {'text': '爱德华·戴维斯·伍德（1924年10月10日至1978年12月10日）是美国电影制片人、演员、作家、制片人和导演。',
     'start': 109,
     'end': 166}],
   'ASPAN': [],
   'ANSWER': [{'ATEXT': '对', 'ATOKEN': {'text': '对', 'start': 0}},
    {'ATEXT': 'yes', 'ATOKEN': {'text': 'yes', 'start': 0}}]}]}